In [2]:
import pytz
import aiohttp
import asyncio
import datetime


async def datetime_to_iso_string(date: datetime.datetime) -> str :
    return (
        date
        .replace(microsecond=0)
        .astimezone(pytz.timezone('Asia/Seoul'))
        .isoformat()
    )


async def request_sampling_data(target_keyword: list, size_per_channel: int, day_interval: int):
    url = 'http://k8s.mysterico.com:31464/analyzer/niz_sample_data/generate'
    async with aiohttp.ClientSession() as session:
        today = datetime.datetime.now()
        iso_today = await datetime_to_iso_string(today)
        interval = datetime.timedelta(days=day_interval)
        response = await session.post(
            url=url,
            json={
                "size_per_channel": size_per_channel,
                "target_keyword": target_keyword,
                "start_date": await datetime_to_iso_string(today - interval),
                "end_date": iso_today,
                "include": ["channel"]
            }
        )
        return await response.json()


async def random_sampling(target_keyword: list, size_per_channel: int, day_interval: int):
    response = await request_sampling_data(target_keyword, size_per_channel, day_interval)
    documents = response.get("documents")
    documents = [document.get("contentPlainText") for document in documents]
    return documents




In [3]:
import unicodedata
from typing import List

async def request_absa(target_keyword: str, sentences: List[str]):
    url = 'http://127.0.0.1:8000/predict_document_sentiment'
    sentences = [unicodedata.normalize('NFC', sentence) for sentence in sentences]
    async with aiohttp.ClientSession() as session:
        response = await session.post(
            url=url,
            json={
                "target_keyword": target_keyword,
                "sentences": sentences
            }
        )
        return await response.json()

In [5]:
await request_absa('비건샴푸', ['''👨‍🔧👩‍🌾🧑‍🌾🧑‍🚀👩‍🚀👨‍🔬야경스타그램 일러스트그램 비건샴푸 성실한 🌚 '''])

[{'sentence': '👨\u200d🔧👩\u200d🌾🧑\u200d🌾🧑\u200d🚀👩\u200d🚀👨\u200d🔬야경스타그램 일러스트그램 비건샴푸 성실한 🌚 ',
  'merged_sentence': '                         [일러스트그램 비건샴푸 : positive] 성실한 🌚',
  'document_sentiment': 'positive'}]

In [14]:
import time
import random

characters_per_line = 100
sampling_trial = 20
target_keyword = "삼쩜삼"

documents = await random_sampling(
    target_keyword=[target_keyword],
    size_per_channel=1000,
    day_interval=2 ** 10
)
random_documents = [random.choice(documents) for _ in range(sampling_trial)]

print(f'total documents : {len(documents)}')
print(f'sampled documents : {len(random_documents)}\n')

def split_documents_by_index(document: str, index: int):
    criterias = [(idx, idx+index) for idx in range(0, len(document), index)]
    result = [document[start_idx:end_idx] for start_idx, end_idx in criterias]
    return '\n'.join(result)

for random_document in random_documents:
    try:
        start = time.time()
        result = await request_absa(target_keyword, [random_document])
        for k, result in result[0].items():
            print(f'<<{k}>>')
            if k == 'merged_sentence':
                print(split_documents_by_index(result, characters_per_line), '\n')
                continue
            print(result, '\n')
        print(time.time() - start, 'sec')
        print('='*100, '\n')
    except Exception as e:
        print('\n', e)
        print(random_document, '\n')
        continue

total documents : 6106
sampled documents : 20

<<sentence>>
#이휘재사과 #층간소음 #문정원
일체 언급없던 이휘재 결국 방송을 통해 언급

영상이 도움이 되셨다면  '구독하기'와 '좋아요' 한번씩 눌러주세요!
댓글은 저에게 매우 큰힘이 된답니다! :D

BGM: Ehrling - Lounge
https://youtu.be/a5ImNzexO4E 

<<merged_sentence>>
#이[휘 : negative]재사과 #층간소음 #[문정원 : negative] 일체 언급없던 [이휘재 : negative] 결국 [방송을 : positive] 통해 [언급  영상 
: positive]이 도움이 되셨다면  '[구독하기 : positive]'와 '[좋아요 : positive]' 한번씩 눌러주세요! [댓글은 : positive] 저에게 매우 큰힘
이 된답니다! :D  BGM: Ehrling - Lounge https://youtu.be/a5ImNzexO4E 

<<document_sentiment>>
neutral 

0.22018814086914062 sec

<<sentence>>
20030303 앱테크 모음 20030303 ・ 35분 전 URL 복사 이웃추가 본문 기타 기능 공유하기 신고하기 더폴 ~10.30 본 링크 가입 시 이디야 스노우쿠키슈 (정가 2,800원) + 500원 + 친구 초대 시 1,000원씩 https://creator.thepol.com/shortlinks/99d5aa07-30e4-4ac5-bbcb-12c957d325e9 빵 소비 미니 트렌드리포트 이벤트🥐 [THEPOL] 블록체인 기반 투표/조사 creator.thepol.com 코인원 가입월 2만원 이상 거래 시 2만원 지급 https://coinone.co.kr/user/signup?ref=BWYNJC48 코인원(Coinone) - 가상자산 거래소의 기준을 제시하다 가상자산 거래는 쉽고 편한 코인원, 비트코인, 이더리움, 클레이튼, 실시간 시세